In [2]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 

import nltk # for stopwords 
from nltk.corpus import stopwords
import gensim # for Word2Vec embeddings 
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
test_content = np.load('test_content_padded.npy')
test_title = np.load('test_title_padded.npy')

In [4]:
import numpy as np 
import pandas as pd 
import re 
import matplotlib.pyplot as plt 
import seaborn as sns

In [31]:
train = pd.read_csv('news_train.csv') 
test = pd.read_csv('news_test.csv')
submission = pd.read_csv('sample_submission.csv')

In [5]:
train.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


In [6]:
train["full_text"] = train['title'] + " " + train['content'] 

In [7]:
train.head()

,n_id,date,title,content,ord,info,full_text
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0,[마감]코스닥 기관 678억 순매도 [이데일리 MARKETPOINT]15:32 현재...
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1,"[마감]코스닥 기관 678억 순매도 ""실적기반"" 저가에 매집해야 할 8월 급등유망주..."
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1,"[마감]코스닥 기관 678억 순매도 하이스탁론, 선취수수료 없는 월 0.4% 최저금..."
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0,[마감]코스닥 기관 678억 순매도 종합 경제정보 미디어 이데일리 - 무단전재 & ...
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참 전국적인 소비 붐 조성에 기여할 예정"


In [8]:
test.head()

,n_id,date,title,content,ord,id
0,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가',1,NEWS00237_1
1,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...,2,NEWS00237_2
2,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,3,NEWS00237_3
3,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,4,NEWS00237_4
4,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대","코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...",5,NEWS00237_5


In [9]:
test['full_text'] = test['title'] + " " + test['content']

In [10]:
test.head()

,n_id,date,title,content,ord,id,full_text
0,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가',1,NEWS00237_1,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대 마이크로 LED T..."
1,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...,2,NEWS00237_2,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대 [이데일리 김대웅 ..."
2,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,3,NEWS00237_3,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대 최근 대형 업체들과..."
3,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,4,NEWS00237_4,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대 TV를 필두로 올해..."
4,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대","코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...",5,NEWS00237_5,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대 코세스는 반도체 장..."


In [5]:
from konlpy.tag import Mecab 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV, train_test_split

In [ ]:
mecab = Mecab() 
def mecab_tokenizer(text): 
    tokens_mecab = mecab.morphs(text) 
    return tokens_mecab

In [ ]:
x_train = train['full_text'] 
y_train = train['info'] 

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

### Multinomial Naive Bayes 

In [ ]:
from sklearn.calibration import CalibratedClassifierCV 
from sklearn.naive_bayes import MultinomialNB 

tfidf_vect = TfidfVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
tfidf_vect.fit(x_train) 
tfidf_matrix_train = tfidf_vect.transform(x_train)

In [ ]:
model = CalibratedClassifierCV(MultinomialNB(alpha=0.03), method='isotonic')
model.fit(tfidf_matrix_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score 
tfidf_matrix_test = tfidf_vect.transform(x_val) 
preds = model.predict(tfidf_matrix_test) 

print("Multinomial Naive Bayes accuracy = ", accuracy_score(y_val, preds))

### Passive Aggressive Classifier

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier 

model = CalibratedClassifierCV(PassiveAggressiveClassifier(C=1e-2, tol=1e-3), method='isotonic')
model.fit(tfidf_matrix_train, y_train)

In [ ]:
preds = model.predict(tfidf_matrix_test)
print("Passive Aggressive Classifier accuracy = ", accuracy_score(y_val, preds))

### Linear Suppor Vector Classifier

In [ ]:
from sklearn.svm import LinearSVC 
model = LinearSVC() 
model.fit(tfidf_matrix_train, y_train)

In [ ]:
preds = model.predict(tfidf_matrix_test)
print("Linear SVC accuracy = ", accuracy_score(y_val, preds))

In [ ]:
probs = model.decision_function(tfidf_matrix_test) 

In [ ]:
pred_probs = (probs - probs.min())/(probs.max() - probs.min()) 

In [ ]:
pred_probs

In [ ]:
# with calibration 
svc_calibrated = CalibratedClassifierCV(LinearSVC(C=2, penalty='l1', max_iter=10000, dual=False),
                                   method='isotonic')
svc_calibrated.fit(tfidf_matrix_train, y_train)

In [ ]:
preds = svc_calibrated.predict(tfidf_matrix_test)
print("Calibrated Linear SVC accuracy = ", accuracy_score(y_val, preds)) 

### Making Prediction

In [11]:
x_train_full = train['full_text'] 
y_train_full = train['info'] 

x_train_full = np.asarray(x_train_full)
y_train_full = np.asarray(y_train_full)

In [12]:
x_train_full.shape, y_train_full.shape

((118745,), (118745,))

In [15]:
mecab = Mecab() 
def mecab_tokenizer(text): 
    tokens_mecab = mecab.morphs(text) 
    return tokens_mecab


tfidf_vect = TfidfVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
tfidf_vect.fit(x_train_full) 
tfidf_matrix_train = tfidf_vect.transform(x_train_full) 


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [17]:
from sklearn.svm import LinearSVC 
model = LinearSVC() 
model.fit(tfidf_matrix_train, y_train_full)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [18]:
x_test = test['full_text'] 
x_test = np.asarray(x_test) 

In [19]:
tfidf_matrix_test = tfidf_vect.transform(x_test) 
preds = model.predict(tfidf_matrix_test)

In [ ]:
submission['info'] = preds 
submission.head()

In [20]:
svc_probs = model.decision_function(tfidf_matrix_test)
pred_probs = (svc_probs - svc_probs.min())/(svc_probs.max() - svc_probs.min())   

In [23]:
np.save('linear_svc_probs.npy',pred_probs) 

In [6]:
pred_probs = np.load('linear_svc_probs.npy')

In [14]:
pred_probs.shape

(142565,)

In [15]:
pred_probs = pred_probs.reshape((142565,1))

In [16]:
pred_probs.shape

(142565, 1)

In [8]:
nn1 = load_model('./bidirectional_nn/m1_epoch_003_val_0.027.h5') 
nn2 = load_model('./bidirectional_nn/m2_epoch_003_val_0.027.h5')
nn3 = load_model('./bidirectional_nn/m3_epoch_002_val_0.030.h5') 
nn4 = load_model('./bidirectional_nn/m4_epoch_002_val_0.024.h5') 
nn5 = load_model('./bidirectional_nn/m5_epoch_002_val_0.021.h5')

In [9]:
pred_nn1 = nn1.predict([test_title, test_content])

In [13]:
pred_nn1.shape

(142565, 1)

In [17]:
pred_nn2 = nn2.predict([test_title, test_content]) 

In [18]:
pred_nn2.shape

(142565, 1)

In [19]:
pred_nn3 = nn3.predict([test_title, test_content]) 

In [20]:
pred_nn3.shape

(142565, 1)

In [21]:
pred_nn4 = nn4.predict([test_title, test_content]) 

In [22]:
pred_nn4.shape 

(142565, 1)

In [23]:
pred_nn5 = nn5.predict([test_title, test_content]) 

In [24]:
pred_nn5.shape 

(142565, 1)

In [26]:
print(type(pred_nn5), type(pred_probs), type(pred_nn1), type(pred_nn2), type(pred_nn3), type(pred_nn4))  

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [27]:
pred_avg = (pred_probs + pred_nn1 + pred_nn2 + pred_nn3 + pred_nn4 + pred_nn5)/6.0 

In [28]:
class_pred = np.where(pred_avg > 0.5, 1, 0).reshape(-1) 

In [29]:
class_pred 

array([0, 0, 0, ..., 1, 1, 1])

In [32]:
submission['info'] = class_pred 
submission.head() 

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0


In [33]:
submission.to_csv('linear_svc_bidirectional_lstm_ensemble.csv',index=False) 